<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

## Getting Started

Let's start importing libraries and creating useful variables 

In [ ]:
%load_ext autotime

In [ ]:
from confluent_kafka import avro
from confluent_kafka.avro import AvroProducer
import time
import qcutils

servers=qcutils.read_config_value("kafka.server") + ":" + str(qcutils.read_config_value("kafka.port"))
sr_url=qcutils.read_config_value("kafka.schema_registry.url")

topic = ''

qcutils.create_kafka_topic(topic)

# write a kafka avro producer

Define avro schemas for keys and value to describe cards in a deck

### Key
The key has a single field to contain arbitrary `Name`

### Value
A card is described by an int `Number` and a `Suit`.

**Note**: The card havo only 4 different suits

1) define the key schema

In [ ]:
key_schema_str = """
    {
        "namespace": "EDV.avro",
        "type": "record",
        "name": "CardKey",
        "fields": [
            {
                "name": "id",
                "type": "int"
            }
        ]
    }
"""

key_schema = avro.loads(key_schema_str)

2) define the value schema

In [ ]:
value_schema_str = """
    {
        "namespace": "EDV.avro",
        "type": "record",
        "name": "CardValue",
        "fields": [
            {
                "name": "numero",
                "type": "int"
            },
            {
                "name": "suit",
                "type": {
                    "type": "enum",
                    "name": "possibleSuits",
                    "symbols": ["C","Q","F","P"]
                }
            }
        ]
    }
"""

value_schema = avro.loads(value_schema_str)

3) create a topic `<surname>-avro-cards` and send in messages such as `SPADES 2`, `HEARTS Q`, `CLUBS K`, ...

In [ ]:
conf = { 
    'bootstrap.servers': servers,
    'schema.registry.url': sr_url
}
p = AvroProducer(conf,
                 default_key_schema=key_schema,
                 default_value_schema=value_schema)

In [ ]:
key = {"id": 2}
value = {"numero": 3, "suit": "F"}

p.produce(topic=topic,
         value=value,
         key=key)

## Schema Evolution

A system that reads the card data, needs the color of the card for a statistical analysis.

Modify the schema to add the card color, and send new data with the card color.

Do you need to change the topic or the consumer implementation?

In [ ]:
new_value_schema_str = """
    {
        "namespace": "EDV.avro",
        "type": "record",
        "name": "CardValue",
        "fields": [
            {
                "name": "numero",
                "type": "int"
            },
            {
                "name": "suit",
                "type": {
                    "type": "enum",
                    "name": "possibleSuits",
                    "symbols": ["C","Q","F","P"]
                }
            },
            {
                "name": "color",
                "type": "string",
                "default": "blue"
            }
        ]
    }
"""

new_value_schema = avro.loads(new_value_schema_str)

In [ ]:
key = {"id": 4}
value = {"numero": 4, "suit": "P", "color": "red"}

p.produce(topic=topic,
         value=value,
         key=key,
         key_schema=key_schema,
         value_schema=new_value_schema)

##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) 2020 Quantia Consulting, srl. All rights reserved.